In [ ]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

import matplotlib.pyplot as plt
import numpy as np

class NN(nn.Module):
    def __init__(self, input_size, output_size):
        super(NN, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 784
output_size = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 3

# Load Training and Test data
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

images, labels = next(iter(train_loader))
print(images.shape)
plt.imshow(images[1].reshape(28,28), cmap="gray")

In [ ]:
# Initialize network
model = NN(input_size=input_size, output_size=output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss() # Good choice for clasifying
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Gradient descent

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Get to correct shape
        data = data.reshape(data.shape[0], -1) # Make images to be 1D vectors

        # forward
        scores = model(data) # We process the value of the NN for the presented data
        loss = criterion(scores, targets) # We compare the score with the expected values

        # backward
        optimizer.zero_grad() # Set to zero the initial conditions
        loss.backward() # Computes the gradient

        # gradient descent or adam step
        optimizer.step()  # Performs the gradient descent


# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval() # Evaluation mode, so it does not do unneccesary calculations

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train() # Come back to training mode
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

In [ ]:
# Let's check the score on a single image

images, label = next(iter(train_loader))
#print(images[0].reshape(28,28))
plt.imshow(images[0].reshape(28,28), cmap="gray")

model.eval()
x = images[0].to(device=device)
score = model(x.reshape(x.shape[0],-1))

print(score)
print(score.max(1))

In [ ]:
from PIL import Image

img = Image.open("three.jpg")

plt.imshow(img, cmap="gray")
print(img.mode)

convert_tensor = transforms.ToTensor()
img_tensor = convert_tensor(img)

x = img_tensor.to(device=device)
score = model(x.reshape(x.shape[0],-1))

print(score)
print(score.max(1))
